### Jupyter Notebook для первой лекции, где мы рассмотрим данные ATLAS Open Data, их структуру и базовые операции с ними. В ноутбуке будет:

1. Чтение ROOT файлов с помощью uproot
2. Просмотр структуры данных
3. Вывод первых строк для ознакомления
4. Базовая визуализация

### First time setup on your computer
This first cell only needs to be run the first time you open this notebook on your computer.
If you close Jupyter and re-open on the same computer, you won't need to run this first cell again.

## To setup everytime

In [ ]:
import urllib.request # for downloading files
import pandas as pd # to store data as dataframes
import numpy as np # for numerical calculations such as histogramming
import uproot3
import uproot
import awkward as ak
import matplotlib.pyplot as plt # for plotting

## Где находятся данные?

В эксперименте ATLAS данные разбиты на несколько файлов в зависимости от **накопленной светимости (luminosity)**. Светимость измеряется в **фемтобарнах (fb⁻¹)** и определяет количество собранных событий за определённый период работы детектора. Чем выше светимость, тем больше данных было записано.

Файлы данных:
- `data_A.exactly2lep.root` → **Светимость 0.5 fb⁻¹**
- `data_B.exactly2lep.root` → **Светимость 1.9 fb⁻¹**
- `data_C.exactly2lep.root` → **Светимость 2.9 fb⁻¹**
- `data_D.exactly2lep.root` → **Светимость 4.7 fb⁻¹**
- Объединённые данные `data_A + data_B + data_C + data_D` → **Светимость 10 fb⁻¹**

🔹 **Что это значит?**
Разные файлы содержат данные, записанные в разное время работы детектора ATLAS. Более поздние данные (например, `data_D`) соответствуют более высокой интегральной светимости, то есть они включают больше столкновений, а значит, потенциально больше интересных событий.

In [ ]:
filename = 'data_A.exactly2lep.root'
url = 'https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/exactly2lep/Data/'+filename
urllib.request.urlretrieve(url, filename)

In [ ]:
file = uproot.open(filename)

In [ ]:
# Проверяем наличие дерева
tree_name = "mini"  # Имя дерева
if tree_name in file:
    tree = file[tree_name]
    print("List of branches:")
    print(tree.keys())
else:
    print("Error: TTree is not found!")

In [ ]:
# Определяем бранчи, относящиеся к лептонам
lep_branches = ['lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_pt', 'lep_eta', 'lep_phi', 'lep_E', 
                'lep_z0', 'lep_charge', 'lep_type', 'lep_isTightID', 'lep_ptcone30', 'lep_etcone20', 
                'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased']

# Читаем данные в Pandas DataFrame
df_leptons = tree.arrays(lep_branches)  # Используем Pandas

In [ ]:
# Выводим первые строки с переменной lep_pt
print("Branch lep_pt in MeV:")
print(df_leptons['lep_pt'])

In [ ]:
# Выводим первые строки с переменной lep_pt
print("Branch lep_pt in GeV:")
print(df_leptons['lep_pt']/1000)

In [ ]:
def plot_distribution(data, column, bins=50, range=None, xlabel=None, ylabel="Number of events", title=None):
    """
    Функция для построения гистограммы заданной переменной из данных.
    
    Parameters:
    - data: DataFrame (Pandas) или Awkward Array
    - column: str, название столбца для построения
    - bins: int, количество бинов в гистограмме
    - range: tuple, диапазон значений (xmin, xmax)
    - xlabel: str, подпись оси X
    - ylabel: str, подпись оси Y (по умолчанию "Number of events")
    - title: str, заголовок графика
    """
    plt.figure(figsize=(8,6))
    
    # Преобразуем в плоский массив, если данные в Awkward Array
    column_data = ak.flatten(data[column]).to_numpy()
    print(column_data)
    
    plt.hist(column_data, bins=bins, range=range, alpha=0.7, color='blue', edgecolor='black')
    plt.xlabel(xlabel if xlabel else column)
    plt.ylabel(ylabel)
    plt.title(title if title else f"Distribution of {column}")
    plt.grid()
    plt.show()

In [ ]:
# Строим гистограмму для lep_pt
plot_distribution(df_leptons, 'lep_pt', bins=50, range=(0, 150000), xlabel=r"Lepton $p_T$ (MeV)", title="Lepton Transverse Momentum Distribution")

In [ ]:
# Выводим первые строки с переменной lep_eta
print("Branch lep_eta:")
print(df_leptons['lep_eta'])

In [ ]:
# Строим гистограмму для lep_eta
plot_distribution(df_leptons, 'lep_eta', bins=50, range=(-3, 3), xlabel=r"Lepton $eta$", title="Lepton $eta$")

In [ ]:
# Выводим первые строки с переменной lep_eta
print("Branch lep_phi:")
print(df_leptons['lep_phi'])

In [ ]:
# Строим гистограмму для lep_pt
plot_distribution(df_leptons, 'lep_phi', bins=50, range=(-4, 4), xlabel=r"Lepton $phi$", title="Lepton $phi$")

In [ ]:
plot_distribution(df_leptons, 'lep_charge', bins=50, range=(-2, 2), xlabel=r"Lepton charge", title="Lepton $charge$")

### А как понять, где мюон, где электрон, а где тау-лептон?

In [ ]:
print("Branch lep_type:")
print(df_leptons['lep_type'])

Checking the PDG ID of particles: https://pdg.lbl.gov/2020/reviews/rpp2020-rev-monte-carlo-numbering.pdf 

In [ ]:
# Строим гистограмму для lep_eta
plot_distribution(df_leptons, 'lep_type', bins=50, range=(10, 20), xlabel=r"Lepton type", title="Lepton type")

In [ ]:
# Create masks for electrons and muons
electron_mask = df_leptons['lep_type'] == 11  # 11 corresponds to electrons
muon_mask = df_leptons['lep_type'] == 13  # 13 corresponds to muons

In [ ]:
# Apply the mask and flatten the arrays
electron_pt = ak.flatten(df_leptons['lep_pt'][electron_mask]).to_numpy()
muon_pt = ak.flatten(df_leptons['lep_pt'][muon_mask]).to_numpy()

In [ ]:
# Convert MeV to GeV if necessary
electron_pt = electron_pt / 1000 if np.median(electron_pt) > 1000 else electron_pt
muon_pt = muon_pt / 1000 if np.median(muon_pt) > 1000 else muon_pt

In [ ]:
# Plot both distributions
plt.figure(figsize=(8,6))
plt.hist(electron_pt, bins=50, range=(0, 150), alpha=0.7, color='red', edgecolor='black', label='Electrons')
plt.hist(muon_pt, bins=50, range=(0, 150), alpha=0.7, color='blue', edgecolor='black', label='Muons')

plt.xlabel(r"$p_T$ (GeV)")
plt.ylabel("Number of leptons")
plt.title("Transverse Momentum Distribution")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Apply the mask and flatten the arrays
electron_eta = ak.flatten(df_leptons['lep_eta'][electron_mask]).to_numpy()
muon_eta = ak.flatten(df_leptons['lep_eta'][muon_mask]).to_numpy()

In [ ]:
# Plot both distributions
plt.figure(figsize=(8,6))
plt.hist(electron_eta, bins=50, range=(-3, 3), alpha=0.7, color='red', edgecolor='black', label='Electrons')
# plt.hist(muon_eta, bins=50, range=(-3, 3), alpha=0.7, color='blue', edgecolor='black', label='Muons')

plt.xlabel(r" eta")
plt.ylabel("Number of electrons")
plt.title("Eta distribution")
plt.legend()
plt.grid()
plt.show()

# Plot both distributions
plt.figure(figsize=(8,6))
# plt.hist(electron_eta, bins=50, range=(-3, 3), alpha=0.7, color='red', edgecolor='black', label='Electrons')
plt.hist(muon_eta, bins=50, range=(-3, 3), alpha=0.7, color='blue', edgecolor='black', label='Muons')

plt.xlabel(r" eta")
plt.ylabel("Number of muons")
plt.title("Eta distribution")
plt.legend()
plt.grid()
plt.show()


### Задание: 

1. Посмотрите на распределение тау-лептонов, и любые другие объекты, которые вам кажутся интересными
2. Сейчас вы работаетe с Data_A, попробуйте посмотреть на MC данные, подумайте, почему результаты отличаются (https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/exactly2lep/MC/mc_361106.Zee.exactly2lep.root, https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/exactly2lep/MC/mc_361107.Zmumu.exactly2lep.root)

<a id='contents'></a>

<a id='running'></a>